# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.core import Workspace, ScriptRunConfig, Environment

## Dataset

In [2]:
ws = Workspace.from_config()
experiment_name = 'Capstone-Project-Experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "hyperdrive-compute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS3_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
env = Environment(name='hyper_env')

# Specify Python dependencies
env.python.conda_dependencies.add_pip_package('pandas')
env.python.conda_dependencies.add_pip_package('scikit-learn')

# Register the environment in your workspace
env.register(workspace=ws)

{
    "assetId": "azureml://locations/westeurope/workspaces/17b33f84-79e4-4c69-b423-0e3018a380a3/environments/hyper_env/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "hyper_env",
    "python"

## Hyperdrive Configuration



In [5]:
# Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.2, delay_evaluation=5)

#Create the different params that you will be using during training
param_sampling = RandomParameterSampling({"--C": choice(1,2,3,4,5), "--max_iter": choice(80,100,120,150,170,200)})

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=trainCluster,
                      environment=env
                      )


hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=param_sampling,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=early_termination_policy,
                                     run_config=src,
                                     max_concurrent_runs=4,
                                     max_total_runs=16,                                     
                                    )

In [6]:
#Submit your experiment
hyperDrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperDrive_run).show()
hyperDrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_00207085-50e3-4d41-8a4f-5a965174b0fc
Web View: https://ml.azure.com/runs/HD_00207085-50e3-4d41-8a4f-5a965174b0fc?wsid=/subscriptions/f5091c60-1c3c-430f-8d81-d802f6bf2414/resourcegroups/aml-quickstarts-269495/workspaces/quick-starts-ws-269495&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-11-05T01:30:45.8083318Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2024-11-05T01:30:46.3066528Z][SCHEDULER][INFO]Scheduling job, id='HD_00207085-50e3-4d41-8a4f-5a965174b0fc_0' 
[2024-11-05T01:30:46.4392826Z][SCHEDULER][INFO]Scheduling job, id='HD_00207085-50e3-4d41-8a4f-5a965174b0fc_2' 
[2024-11-05T01:30:46.4406147Z][SCHEDULER][INFO]Scheduling job, id='HD_00207085-50e3-4d41-8a4f-5a965174b0fc_1' 
[2024-11-05T01:30:46.5617483Z][SCHEDULER][INFO]Scheduling job, id='HD_00207085-50e3-4d41-8a4f-5a965174b0fc_3' 
[2024-11-05T01:30:46.7345537Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_00207085-50e3-4d41-8a4f-5a965174b0fc_2' 
[2024-11-05T01:3

{'runId': 'HD_00207085-50e3-4d41-8a4f-5a965174b0fc',
 'target': 'hyperdrive-compute',
 'status': 'Completed',
 'startTimeUtc': '2024-11-05T01:30:44.297707Z',
 'endTimeUtc': '2024-11-05T01:48:24.422249Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '159540ed-83e6-4db3-8f4a-f2c90f5af92f',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_00207085-50e3-4d41-8a4f-5a965174b0fc_9',
  'score': '0.8757037037037037',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_00207085-50e3-4d41-8a4f-5a965174b0fc_9'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azurem

## Best Model

In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
best_run = hyperDrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Regularization Strength:',best_run_metrics['Regularization Strength:'])
print('\n Max Iterations:',best_run_metrics['Max iterations:'])

Best Run Id:  HD_00207085-50e3-4d41-8a4f-5a965174b0fc_9

 Accuracy: 0.8757037037037037

 Regularization Strength: 4.0

 Max Iterations: 200


In [12]:
# Code below registers the best model with the information of Metrics
model = best_run.register_model(model_name='HyperDrive_HighAccuracy', model_path='outputs/', 
                                properties={'Accuracy': best_run_metrics['Accuracy'],
                                            'Regularization Strength': best_run_metrics['Regularization Strength:'],
                                           'Max Iterations': best_run_metrics['Max iterations:']})